In [7]:
import datetime
import os

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf

from tensorboard.plugins.hparams import api as hp



In [8]:
params_grid = {
    "embedder_data" : "cc.fr.300.bin",
    "dataset": "fr_hf.csv"
}


In [18]:
data_folder_src = './../../data/processed/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'

vector_path = f'./../../embedded_vector/{params_grid["embedder_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'


ft = fasttext.load_model(f'../../data/embedding_data/{params_grid["embedder_data"]}')


In [19]:

ds = pd.read_csv(data_path_src)
ds

,text,labels,target,source,dataset,nb_annotators,tweet_id
0,On ne peut pas faire confiance aux musulmans c...,1,islam,NGO,CONAN,1,NaN
1,Les musulmans ne sont pas dignes de confiance.,1,islam,NGO,CONAN,1,NaN
2,Vous ne devriez pas croire les musulmans.,1,islam,NGO,CONAN,1,NaN
3,Les musulmans empêchent les gens de manger pen...,1,islam,NGO,CONAN,1,NaN
4,Les musulmans ne laissent pas les gens s'en pr...,1,islam,NGO,CONAN,1,NaN
...,...,...,...,...,...,...,...
18066,Kevin câest un sal noir.,1,NaN,Twitter,FTR,2,NaN
18067,"bougnoule bamboula sale arabe , elle est belle...",1,NaN,Twitter,FTR,2,NaN
18068,@USER Tu viens de le traiter de sal noir ?,1,NaN,Twitter,FTR,2,NaN
18069,SAL NOIR LINK,1,NaN,Twitter,FTR,2,NaN


In [20]:

def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)


In [21]:
tqdm.pandas(desc='Loading data')
ds['vector'] = ds['text'].progress_apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['labels'].values


Loading data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18071/18071 [26:11<00:00, 11.50it/s]


In [22]:
np.save(vector_x_dst, X)
np.save(vector_y_dst, y)